In [1]:
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Flatten
from sklearn.metrics import accuracy_score
from keras import metrics

Using TensorFlow backend.


In [2]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [3]:
# load dataset
dataset = read_csv('F:\\New folder\\project_stock\\cl1_oil_2000.csv',header=0)
print(dataset.columns.values.tolist())
dataset.drop(['Unnamed: 0', 'Date', 'Open', 'High', 'Low', 'Volume', 'sma', 'rsi', 'trend'], axis=1, inplace=True)
values = dataset.values
values

['Unnamed: 0', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'macd', 'macd_signal', 'sma', 'rsi', 'trend']


array([[25.55      ,  0.35688551,  1.53974239],
       [24.91      , -0.70034465,  1.09172498],
       [24.78      , -1.47122329,  0.57913532],
       ...,
       [50.94      ,  0.6270941 ,  1.38982244],
       [51.02      ,  0.95813397,  1.30348474],
       [50.74      ,  0.9933803 ,  1.24146385]])

In [4]:
# ensure all data is float
values = values.astype('float32')
values

array([[25.55      ,  0.35688552,  1.5397424 ],
       [24.91      , -0.7003446 ,  1.091725  ],
       [24.78      , -1.4712234 ,  0.5791353 ],
       ...,
       [50.94      ,  0.6270941 ,  1.3898225 ],
       [51.02      ,  0.95813394,  1.3034848 ],
       [50.74      ,  0.9933803 ,  1.2414639 ]], dtype=float32)

In [5]:
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
scaled

array([[0.06336045, 0.62625265, 0.678324  ],
       [0.0583542 , 0.59376156, 0.6622493 ],
       [0.0573373 , 0.5700707 , 0.64385784],
       ...,
       [0.26196808, 0.6345568 , 0.67294496],
       [0.26259387, 0.6447304 , 0.6698472 ],
       [0.26040363, 0.6458136 , 0.6676219 ]], dtype=float32)

In [6]:
scaled.shape

(4470, 3)

In [7]:
# frame as supervised learning
import numpy as np
reframed = series_to_supervised(scaled, 3, 1)
#np.array(reframed)
rd=reframed.values
print(reframed.head())
#type(rd)

   var1(t-3)  var2(t-3)  var3(t-3)  var1(t-2)  var2(t-2)  var3(t-2)  \
3   0.063360   0.626253   0.678324   0.058354   0.593762   0.662249   
4   0.058354   0.593762   0.662249   0.057337   0.570071   0.643858   
5   0.057337   0.570071   0.643858   0.052957   0.538914   0.621870   
6   0.052957   0.538914   0.621870   0.056477   0.535485   0.603478   
7   0.056477   0.535485   0.603478   0.065081   0.569837   0.596787   

   var1(t-1)  var2(t-1)  var3(t-1)   var1(t)   var2(t)   var3(t)  
3   0.057337   0.570071   0.643858  0.052957  0.538914  0.621870  
4   0.052957   0.538914   0.621870  0.056477  0.535485  0.603478  
5   0.056477   0.535485   0.603478  0.065081  0.569837  0.596787  
6   0.065081   0.569837   0.596787  0.069071  0.608054  0.600357  
7   0.069071   0.608054   0.600357  0.072278  0.644433  0.611707  


In [8]:
values = reframed.values
n_train_records = int(rd.shape[0] * .80)
train = values[:n_train_records, :]
test = values[n_train_records:, :]
print(values.shape)
print (train.shape[0], test.shape[0])

(4467, 12)
3573 894


In [9]:

# split into input and outputs
train_X, train_y = train[:, :-3], train[:, 9:12]
test_X, test_y = test[:, :-3], test[:, 9:12]

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 3, 3))
test_X = test_X.reshape((test_X.shape[0], 3, 3))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(3573, 3, 3) (3573, 3) (894, 3, 3) (894, 3)


In [10]:
model = Sequential()  
#model.add(Flatten())
#model.add(Dense(4))
model.add(LSTM(3, input_shape=train_X.shape[1:],return_sequences=True,activation='softmax'))
#model.add(LSTM(3, input_shape=train_X.shape[1:],return_sequences=True,activation='softmax'))
model.add(LSTM(3, input_shape=train_X.shape[1:], return_sequences=False,activation='relu'))
model.compile(loss='mse',optimizer='adam',metrics=['accuracy'])

model.summary()
print ("Inputs: {}".format(model.input_shape))
print ("Outputs: {}".format(model.output_shape))
print ("Actual input: {}".format(train_X.shape))
print ("Actual output: {}".format(train_y.shape))

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 3, 3)              84        
_________________________________________________________________
lstm_2 (LSTM)                (None, 3)                 84        
Total params: 168
Trainable params: 168
Non-trainable params: 0
_________________________________________________________________
Inputs: (None, 3, 3)
Outputs: (None, 3)
Actual input: (3573, 3, 3)
Actual output: (3573, 3)


In [11]:
model.fit(train_X, train_y, nb_epoch=50, batch_size=18, validation_data=(test_X,test_y))

Instructions for updating:
Use tf.cast instead.


c:\users\shilpi\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Train on 3573 samples, validate on 894 samples
Epoch 1/50
3573/3573 [==============================] - 6s 2ms/step - loss: 0.2445 - acc: 0.1811 - val_loss: 0.1470 - val_acc: 0.4664
Epoch 2/50
3573/3573 [==============================] - 1s 373us/step - loss: 0.1589 - acc: 0.5250 - val_loss: 0.1377 - val_acc: 0.4664
Epoch 3/50
3573/3573 [==============================] - 1s 418us/step - loss: 0.1557 - acc: 0.5250 - val_loss: 0.1358 - val_acc: 0.4664
Epoch 4/50
3573/3573 [==============================] - 1s 355us/step - loss: 0.1534 - acc: 0.5250 - val_loss: 0.1341 - val_acc: 0.4664
Epoch 5/50
3573/3573 [==============================] - 2s 436us/step - loss: 0.1497 - acc: 0.5250 - val_loss: 0.1313 - val_acc: 0.4664
Epoch 6/50
3573/3573 [==============================] - 2s 426us/step - loss: 0.1444 - acc: 0.5250 - val_loss: 0.1281 - val_acc: 0.4664
Epoch 7/50
3573/3573 [==============================] - 2s 453us/step - loss: 0.1407 - acc: 0.5410 - val_loss: 0.1268 - val_acc: 0.4664
Epo

In [12]:
model.save('F:\\New folder\\project_stock\\cl1_oil_model.h5')

In [13]:
savedModel = load_model('F:\\New folder\\project_stock\\cl1_oil_model.h5')

In [14]:
#savedModel.evaluate(test_X, test_y, batch_size=21, verbose=0, sample_weight=None)
print(test_X[3:4,:,:])
print(test_X[4:5,:,:])
result = savedModel.predict(test_X[5:6,:,:])
print(test_X[5:6,:,:])
print(test_X[6:7,:,:])
print(result)

[[[0.65879226 0.63137823 0.6189242 ]
  [0.65808827 0.6366461  0.6247429 ]
  [0.64369524 0.62477356 0.6266257 ]]]
[[[0.65808827 0.6366461  0.6247429 ]
  [0.64369524 0.62477356 0.6266257 ]
  [0.6427566  0.6162519  0.6261421 ]]]
[[[0.64369524 0.62477356 0.6266257 ]
  [0.6427566  0.6162519  0.6261421 ]
  [0.6479975  0.61629003 0.62576413]]]
[[[0.6427566  0.6162519  0.6261421 ]
  [0.6479975  0.61629003 0.62576413]
  [0.65464646 0.62303936 0.62703776]]]
[[0.6488119 0.        0.6250431]]
